# Explore data files on local disk

In [3]:
import pandas as pd
import numpy

In [6]:
dataroot = "/data/graph-data/northwind/data"
orders = pd.read_csv(dataroot + "/orders.csv")
order_details = pd.read_csv(dataroot + "/order-details.csv")
customers = pd.read_csv(dataroot + "/customers.csv")
reps = pd.read_csv(dataroot + "/employees.csv")

In [7]:
order_details.dtypes


OrderID                  int64
ProductID                int64
UnitPrice              float64
Quantity                 int64
Discount               float64
LineTotalNoDiscount    float64
LineTotal              float64
dtype: object

In [8]:
order_details.describe().round()

,OrderID,ProductID,UnitPrice,Quantity,Discount,LineTotalNoDiscount,LineTotal
count,2155.0,2155.0,2155.0,2155.0,2155.0,2155.0,2155.0
mean,10659.0,41.0,26.0,24.0,0.0,629.0,587.0
std,241.0,22.0,30.0,19.0,0.0,1036.0,969.0
min,10248.0,1.0,2.0,1.0,0.0,5.0,5.0
25%,10451.0,22.0,12.0,10.0,0.0,154.0,147.0
50%,10657.0,41.0,18.0,20.0,0.0,360.0,338.0
75%,10862.0,60.0,32.0,30.0,0.0,722.0,656.0
max,11077.0,77.0,264.0,130.0,0.0,15810.0,15810.0


In [9]:
order_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   OrderID              2155 non-null   int64  
 1   ProductID            2155 non-null   int64  
 2   UnitPrice            2155 non-null   float64
 3   Quantity             2155 non-null   int64  
 4   Discount             2155 non-null   float64
 5   LineTotalNoDiscount  2155 non-null   float64
 6   LineTotal            2155 non-null   float64
dtypes: float64(4), int64(3)
memory usage: 118.0 KB


In [10]:
order_details.head(20)

,OrderID,ProductID,UnitPrice,Quantity,Discount,LineTotalNoDiscount,LineTotal
0,10248,11,14.0,12,0.00,168.0,168.00
1,10248,42,9.8,10,0.00,98.0,98.00
2,10248,72,34.8,5,0.00,174.0,174.00
3,10249,14,18.6,9,0.00,167.4,167.40
4,10249,51,42.4,40,0.00,1696.0,1696.00
5,10250,41,7.7,10,0.00,77.0,77.00
6,10250,51,42.4,35,0.15,1484.0,1261.40
7,10250,65,16.8,15,0.15,252.0,214.20
8,10251,22,16.8,6,0.05,100.8,95.76
9,10251,57,15.6,15,0.05,234.0,222.30


In [11]:
customers['Region'].unique()

array([nan, 'BC', 'SP', 'Lille', 'OR', 'DF', 'RJ', 'Táchira', 'Co. Cork',
       'Isle of Wight', 'WA', 'CA', 'Lara', 'Nueva Esparta', 'Québec',
       'AK', 'NM', 'ID', 'WY', 'MT'], dtype=object)

In [12]:
%%bash
# rm -fv orders.db

In [13]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   OrderID         830 non-null    int64  
 1   CustomerID      830 non-null    object 
 2   EmployeeID      830 non-null    int64  
 3   OrderDate       830 non-null    object 
 4   RequiredDate    830 non-null    object 
 5   ShippedDate     809 non-null    object 
 6   ShipVia         830 non-null    int64  
 7   Freight         830 non-null    float64
 8   ShipName        830 non-null    object 
 9   ShipAddress     830 non-null    object 
 10  ShipCity        830 non-null    object 
 11  ShipRegion      323 non-null    object 
 12  ShipPostalCode  811 non-null    object 
 13  ShipCountry     830 non-null    object 
 14  OrderAmt        830 non-null    float64
 15  CommissionAmt   161 non-null    float64
dtypes: float64(3), int64(3), object(10)
memory usage: 103.9+ KB


In [14]:
orders[orders['CommissionAmt'].notnull()]

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry,OrderAmt,CommissionAmt
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,NaN,51100,France,440.0000,22.00
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.30,Suprêmes délices,Boulevard Tirou 255,Charleroi,NaN,B-6000,Belgium,3597.9000,179.90
10,10258,ERNSH,1,1996-07-17,1996-08-14,1996-07-23,1,140.51,Ernst Handel,Kirchgasse 6,Graz,NaN,8010,Austria,1614.8800,80.74
15,10263,ERNSH,9,1996-07-23,1996-08-20,1996-07-31,3,146.06,Ernst Handel,Kirchgasse 6,Graz,NaN,8010,Austria,1873.8000,93.69
21,10269,WHITC,5,1996-07-31,1996-08-14,1996-08-09,1,4.56,White Clover Markets,1029 - 12th Ave. S.,Seattle,WA,98124,USA,642.2000,32.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,11067,DRACD,1,1998-05-04,1998-05-18,1998-05-06,2,7.98,Drachenblut Delikatessen,Walserweg 21,Aachen,NaN,52066,Germany,86.8500,4.34
820,11068,QUEEN,8,1998-05-04,1998-06-01,NaN,2,81.75,Queen Cozinha,Alameda dos Canàrios 891,Sao Paulo,SP,05487-020,Brazil,2027.0800,101.35
821,11069,TORTU,1,1998-05-04,1998-06-01,1998-05-06,2,15.67,Tortuga Restaurante,Avda. Azteca 123,México D.F.,NaN,5033,Mexico,360.0000,18.00
824,11072,ERNSH,4,1998-05-05,1998-06-02,NaN,2,258.64,Ernst Handel,Kirchgasse 6,Graz,NaN,8010,Austria,5218.0000,260.90


In [15]:
# conn = sqlite3.connect('orders.db')
# orders.to_sql('orders', conn, if_exists='replace')
# order_details.to_sql('order_details', conn, if_exists='replace')
# conn.close()

In [16]:
reps['BirthDate']

0      1948-12-08 00:00:00
1      1952-02-19 00:00:00
2      1963-08-30 00:00:00
3      1937-09-19 00:00:00
4      1955-03-04 00:00:00
              ...         
104    1976-10-29 01:38:40
105    1983-09-18 18:23:26
106    1971-05-21 07:24:41
107    1999-04-30 21:15:09
108    1954-11-21 10:28:30
Name: BirthDate, Length: 109, dtype: object

# Connect to the Northwind Graph using PyTigerGraph

In [17]:
# import sqlite3
from configparser import ConfigParser

import pandas as pd
# Set up pyTigerGraph access
import pyTigerGraph as tg

In [ ]:
# host = "http://medsales.34.106.148.222.nip.io:14240"
parser = ConfigParser()
_ = parser.read('CREDENTIALS.config') # contains DEFAULT section plus override sections

host = parser.get('k3s-lab', 'host')
username = parser.get('k3s-lab', 'user')
password = parser.get('k3s-lab', 'password')
gsPort = parser.get('k3s-lab', 'gsPort')
apiPort = parser.get('k3s-lab', 'apiPort')
mysecret = parser.get('k3s-lab', 'secret')
# token = parser.get('k3s-lab', 'token')
graphName = 'Northwind'


conn = tg.TigerGraphConnection(host=host, restppPort=apiPort, gsPort=gsPort, graphname=graphName, password=password)
token = conn.getToken(mysecret, setToken=True, lifetime=None)[0]

# Next use the new token to establish a full access connection for use with GSQL

# conn = tg.TigerGraphConnection(host=host, restppPort=apiPort, gsPort=gsPort, graphname=graphName, password=password, apiToken=token)
token

In [ ]:
apiPort

In [ ]:
token[0]

In [ ]:
conn.getVertexTypes()

In [ ]:
# conn.getVertexTypes()
# conn.runInstalledQuery('floors')
print(conn.gsql('show edge *', options=[]))

In [ ]:
conn.gsql('drop graph gort', options=[])

In [ ]:
print(conn.gsql('show graph *', options=[]))

# Reps and customers